In [60]:
%%writefile question3b.py
# required modules
import numpy as np
import random
import gym
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from itertools import count
import matplotlib.pyplot as plt

# parsing the arguments
import argparse as arg
arg_parser = arg.ArgumentParser()
arg_parser.add_argument("--environment",type=str)
arg_parser.add_argument("--iterations",type=int)
arg_parser.add_argument("--batch",type=int)
arg_parser.add_argument("--learning_rate",type=float)
arg_parser.add_argument("--gamma",type=float)
arg_parser.add_argument("--reward_to_go",action="store_true")
arg_parser.add_argument("--advantage_norm",action="store_true")


# function to create plot
def plot(rewards):
    plt.figure(figsize=(10,5),dpi=100) 

    plt.title(file_name)
    plt.plot(rewards,label='Mean Reward')
    plt.xlabel('Episodes')
    plt.ylabel('Reward')
    plt.grid()
    plt.savefig(file_name)

# define the policy
class Policy(nn.Module):
    def __init__(self, observation_dimension, action_dimension):
        super(Policy, self).__init__()
        hidden_dimension = 256
        self.model = nn.Sequential(nn.Linear(observation_dimension, hidden_dimension, bias=False),
                                   nn.ReLU(),
                                   nn.Linear(hidden_dimension,action_dimension,bias=False),
                                   nn.Softmax(dim=-1))
        
    def forward(self,x):
        return self.model(x)

# select the action
def select_action(state):
    network = policy_network(state.to(device))
    network = Categorical(network)
    action = network.sample()
    return action, network.log_prob(action)

# fetch the current state
def get_state(obs):
    state = np.array(obs)
    state = torch.from_numpy(state)
    state = state.type(torch.FloatTensor)
    state = state.unsqueeze(0)
    return state

def optimize_model(policy_history_batch,episode_rewards_batch):
    if arguments.advantage_norm:
        const_baseline = 0.
        for episode_rewards in episode_rewards_batch:
            R = 0
            rewards = []
            # Discount future rewards back to the present using gamma
            for r in episode_rewards[::-1]:
                R = r + gamma * R
            const_baseline += R
        const_baseline /= batch_size

    loss = 0.
    for i in range(len(episode_rewards_batch)):
        policy_history = policy_history_batch[i]
        episode_rewards = episode_rewards_batch[i]
        R = 0
        rewards = []
        # Discount future rewards back to the present using gamma
        for r in episode_rewards[::-1]:
            R = r + gamma * R
            rewards.insert(0,R)
            
        # Scale rewards
        policy_history = torch.cat(policy_history).type(torch.FloatTensor)
        rewards = torch.FloatTensor(rewards)
        
        # Calculate loss
        if arguments.reward_to_go:
            if arguments.adv_norm:
                rewards =  rewards - const_baseline
                rewards = (rewards - rewards.mean()) / (rewards.std() + np.finfo(np.float32).eps)

            loss += torch.sum(torch.mul(policy_history,rewards).mul(-1), -1)
        else:
            loss += torch.sum(torch.mul(policy_history,rewards[0]).mul(-1), -1)
    
    # Update network weights
    optimizer.zero_grad()
    loss.backward()
    # print(policy_net.model[0].weight.grad)
    optimizer.step()

# training in the environment
def train(env):
    rewards = []
    it=0
    while it < iterations:   
        episode_rewards_batch = []
        policy_history_batch = []
        episode = 0
        while episode < batch_size:
            obs = env.reset()
            state = get_state(obs)
            episode_rewards = []
            policy_history = []
            for t in count():
                action, policy_log = select_action(state)
                action = action.item()
                a = env.step(action)
                obs = a[0]
                reward = a[1]
                done = a[2]
                recall=[]
                state  =  get_state(obs)
                episode_rewards.append(reward)
                policy_history.append(policy_log)

                if done:
                    break
            episode_rewards_batch.append(episode_rewards)
            policy_history_batch.append(policy_history)
            episode+=1
        rewards.append(np.mean([np.sum(er) for er in episode_rewards_batch]))
        optimize_model(policy_history_batch,episode_rewards_batch)
        print('Total Reward for Iteration {} : {}'.format(it,rewards[-1]))
        it+=1    
    env.close()
    plot(rewards)
    return

# set device
device = torch.device("cpu")

# extract the given arguments
arguments = arg_parser.parse_args()

# Set the Hyperparameters
gamma = arguments.gamma
batch_size = arguments.batch
learning_rate = arguments.learning_rate
iterations = arguments.iterations
environment = gym.make(arguments.environment)

# creating file name to save the plot
file_name=arguments.environment+'_'+str(iterations)+'_'+str(batch_size)+'.png'

# create networks
policy_network = Policy(environment.observation_space.shape[0],environment.action_space.n).to(device)

# setup optimizer
optimizer = optim.Adam(policy_network.parameters(),lr=learning_rate)

# training in the environment
train(environment)

Overwriting question3b.py


In [58]:
!pip install box2d-py

In [61]:
!python3 question3b.py --environment="CartPole-v0" --iterations=50 --batch=32 --gamma=0.9 --learning_rate=0.01

Total Reward for Iteration 0 : 25.15625
Total Reward for Iteration 1 : 28.40625
Total Reward for Iteration 2 : 21.46875
Total Reward for Iteration 3 : 22.15625
Total Reward for Iteration 4 : 18.09375
Total Reward for Iteration 5 : 16.1875
Total Reward for Iteration 6 : 16.3125
Total Reward for Iteration 7 : 14.375
Total Reward for Iteration 8 : 13.5625
Total Reward for Iteration 9 : 13.5
Total Reward for Iteration 10 : 14.53125
Total Reward for Iteration 11 : 15.03125
Total Reward for Iteration 12 : 15.34375
Total Reward for Iteration 13 : 14.25
Total Reward for Iteration 14 : 11.4375
Total Reward for Iteration 15 : 13.375
Total Reward for Iteration 16 : 12.46875
Total Reward for Iteration 17 : 16.1875
Total Reward for Iteration 18 : 12.75
Total Reward for Iteration 19 : 14.78125
Total Reward for Iteration 20 : 13.59375
Total Reward for Iteration 21 : 13.84375
Total Reward for Iteration 22 : 13.75
Total Reward for Iteration 23 : 14.40625
Total Reward for Iteration 24 : 15.53125
Total R

In [62]:
!python3 question3b.py --environment="LunarLander-v2" --iterations=50 --batch=32 --gamma=0.9 --learning_rate=0.01

Total Reward for Iteration 0 : -163.83474827321965
Total Reward for Iteration 1 : -226.92567886446676
Total Reward for Iteration 2 : -298.8460421292288
Total Reward for Iteration 3 : -291.61552929592875
Total Reward for Iteration 4 : -258.98958296821127
Total Reward for Iteration 5 : -209.8683482207271
Total Reward for Iteration 6 : -197.8286535038273
Total Reward for Iteration 7 : -189.22223528562176
Total Reward for Iteration 8 : -154.82292080294164
Total Reward for Iteration 9 : -159.56684465912485
Total Reward for Iteration 10 : -142.4020348832291
Total Reward for Iteration 11 : -150.35297496014508
Total Reward for Iteration 12 : -204.03626635954444
Total Reward for Iteration 13 : -175.62587823660164
Total Reward for Iteration 14 : -165.92619151367307
Total Reward for Iteration 15 : -171.21376882812672
Total Reward for Iteration 16 : -188.90197274977143
Total Reward for Iteration 17 : -216.05342321174953
Total Reward for Iteration 18 : -209.8427269685871
Total Reward for Iteration 